In [1]:
import pandas as pd
from app import app, db, Reservation, DeviceStatus, User
from datetime import datetime, timedelta


In [2]:
xls = pd.ExcelFile('data/dataset.xlsx')
df1 = pd.read_excel(xls, 'Formulierreacties 1')

In [3]:
df1 = df1.drop(['Time of visit', 'Date of visit ', 'Free space for comments', 'Which game are you going to play?', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12','Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'], axis=1)

In [4]:
df1.columns

Index(['Tijdstempel', 'Full name', 'Student number',
       'What PC/Console will you use?',
       'Which university are you from? (If not from a uni, please let us know from where at the "other" section)',
       'E-mailadres'],
      dtype='object')

In [5]:
with app.app_context():
    # Query all reservations
    reservations = Reservation.query.all()
    print("Reservations:", reservations)

    # Query all device statuses
    device_statuses = DeviceStatus.query.all()
    print("Device Statuses:", device_statuses)

    # Query all users
    users = User.query.all()
    print("Users:", users)

    # Query reservations in the last 24 hours
    end_date = datetime.now()
    start_date = end_date - timedelta(days=1)
    reservations_last_24_hours = Reservation.query.filter(
        Reservation.date.between(start_date, end_date)
    ).all()
    print("Reservations in Last 24 Hours:", reservations_last_24_hours)

Reservations: [<Reservation 1 | Name: a | School: BUas | Email: a@email.com | Device: PC1 | Date: 2024-09-04 08:16:49.769020>, <Reservation 2 | Name: b | School: BUas | Email: a@email.com | Device: PC2 | Date: 2024-09-04 08:16:54.625392>, <Reservation 3 | Name: c | School: BUas | Email: a@email.com | Device: PC3 | Date: 2024-09-04 08:16:59.780324>, <Reservation 4 | Name: d | School: BUas | Email: a@email.com | Device: PC4 | Date: 2024-09-04 08:17:03.919225>]
Device Statuses: [<DeviceStatus 1 | Device: PC1 | Status: Reserved>, <DeviceStatus 2 | Device: PC2 | Status: Reserved>, <DeviceStatus 3 | Device: PC3 | Status: Reserved>, <DeviceStatus 4 | Device: PC4 | Status: Reserved>]
Users: [<User 1 | Username: admin>]
Reservations in Last 24 Hours: [<Reservation 1 | Name: a | School: BUas | Email: a@email.com | Device: PC1 | Date: 2024-09-04 08:16:49.769020>, <Reservation 2 | Name: b | School: BUas | Email: a@email.com | Device: PC2 | Date: 2024-09-04 08:16:54.625392>, <Reservation 3 | Name: 

In [6]:
reservations[0]

<Reservation 1 | Name: a | School: BUas | Email: a@email.com | Device: PC1 | Date: 2024-09-04 08:16:49.769020>